In [ ]:
# 用于missing column
import numpy as np
from pdb import lasti2lineno
from pandas.io.parsers import readers
import pandas as pd
import csv

original_name = 'RPunch1415_Original.csv'

with open(original_name) as f:
  reader1 = csv.reader(f)
  lst1 = list(reader1)
  original = np.array(lst1)

original = np.delete(original,0,0)
original = np.delete(original,0,1)
raw_data1 = original.copy()

raw_data1[0:194,4] = 0

TEhalf = raw_data1.copy()
TEhalf = TEhalf.astype(float)
print(TEhalf)

[[3.43116738 5.61843801 4.31186282 6.17209488 0.        ]
 [4.59654273 6.92436691 6.02924363 6.87311132 0.        ]
 [4.27511409 6.2837416  5.98534499 5.95801168 0.        ]
 ...
 [3.30319918 5.49826874 6.24239068 5.55535815 3.27583724]
 [4.64099687 5.80630477 6.13589483 6.27387651 3.5027716 ]
 [4.52790166 5.92684055 5.98113295 5.67102763 3.61363472]]


In [ ]:
IBCF_error_doc = []

In [ ]:
import numpy as np
from pdb import lasti2lineno
from pandas.io.parsers import readers
import pandas as pd
import csv

original_name = 'RPunch1415_Original.csv'
yhat1_name = 'asymPunch1415_c6.csv'

with open(original_name) as f:
  reader1 = csv.reader(f)
  lst1 = list(reader1)
  original = np.array(lst1)


with open(yhat1_name) as f:
  reader21 = csv.reader(f)
  lst21 = list(reader21)
  yhat1 = np.array(lst21)


original = np.delete(original,0,0)
original = np.delete(original,0,1)
yhat1 = np.delete(yhat1,0,0)
yhat1 = np.delete(yhat1,0,1)

raw_data1 = yhat1.copy()
#for i in range(original.shape[0]):
#  for j in range(original.shape[1]):
#    if yhat1[i,j] == "0":
#      raw_data1[i,j] = 0

TEhalf = raw_data1.copy()
TEhalf = TEhalf.astype(float)
print(TEhalf)

[[3.43116738 5.61843801 4.31186282 6.17209488 3.13428758]
 [4.59654273 0.         6.02924363 0.         0.        ]
 [4.27511409 6.2837416  5.98534499 5.95801168 0.        ]
 ...
 [3.30319918 5.49826874 6.24239068 5.55535815 3.27583724]
 [0.         5.80630477 6.13589483 6.27387651 3.5027716 ]
 [4.52790166 5.92684055 5.98113295 5.67102763 3.61363472]]


In [ ]:
from scipy import stats
import statistics

means = [0] * TEhalf.shape[1]
sd = [0] * TEhalf.shape[1]
scaled = np.zeros([TEhalf.shape[0],TEhalf.shape[1]])

for i in range(TEhalf.shape[1]):
  means[i] = np.mean([num for num in TEhalf[:,i] if num!= 0])
  sd[i] = statistics.stdev([num for num in TEhalf[:,i] if num!= 0])

for i in range(TEhalf.shape[1]):
  for j in range(TEhalf.shape[0]):
    if TEhalf[j,i] != 0:
      scaled[j,i] = (TEhalf[j,i] - means[i]) / sd[i]

print(scaled)

[[-2.26481066 -0.94774583 -2.89730609  0.55682938  0.26569821]
 [ 1.23859856  0.          0.09674529  0.          0.        ]
 [ 0.27230383  0.76601177  0.02021318  0.05500425  0.        ]
 ...
 [-2.64951504 -1.25729023  0.46834209 -0.88884218  0.59280502]
 [ 0.         -0.46381926  0.28267911  0.79541225  1.11722716]
 [ 1.03224638 -0.15333075  0.01286998 -0.61770528  1.37342052]]


In [ ]:
from scipy.stats import pearsonr
simTE = np.zeros([scaled.shape[1],scaled.shape[1]])

def calcSim(col1,col2,scaled):
  temp = np.zeros([scaled.shape[0],2])
  temp[:,0] = scaled[:,col1]
  temp[:,1] = scaled[:,col2]
  i = 0
  while i < temp.shape[0]:
    for j in range(temp.shape[1]):
      if temp[i,j] == 0:
        #temp[i,:] = 0
        temp = np.delete(temp,i,0)
        i = i - 1
    i = i + 1
  sim = pearsonr(temp[:,0], temp[:,1])
  return sim[0]

for i in range(scaled.shape[1]):
  for j in range(i,scaled.shape[1]):
    simTE[i,j] = calcSim(i,j,scaled)
    simTE[j,i] = calcSim(i,j,scaled)

print(simTE)

[[1.         0.33705344 0.2769528  0.30326335 0.15687425]
 [0.33705344 1.         0.40654224 0.484991   0.18676053]
 [0.2769528  0.40654224 1.         0.29586517 0.20416376]
 [0.30326335 0.484991   0.29586517 1.         0.18058127]
 [0.15687425 0.18676053 0.20416376 0.18058127 1.        ]]


In [ ]:
zhat = np.zeros([scaled.shape[0],scaled.shape[1]])

def calcZhat(row,col,simTE,scaled):
  upper = 0
  lower = 0
  for i in range(scaled.shape[1]):
    if i != col:
      upper = upper + scaled[row,i] * simTE[col,i]
      lower = lower + abs(simTE[col,i])
  return upper/lower

for i in range(scaled.shape[0]):
  for j in range(scaled.shape[1]):
    if scaled[i,j] == 0:
      zhat[i,j] = calcZhat(i,j,simTE,scaled)

print(zhat)

[[0.         0.         0.         0.         0.        ]
 [0.         0.32275116 0.         0.31963695 0.29387979]
 [0.         0.         0.         0.         0.27435928]
 ...
 [0.         0.         0.         0.         0.        ]
 [0.31507926 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


In [ ]:
for i in range(scaled.shape[0]):
  for j in range(scaled.shape[1]):
    if TEhalf[i,j] == 0:
      TEhalf[i,j] = zhat[i,j] * sd[j] + means[j]
print(TEhalf)

[[3.43116738 5.61843801 4.31186282 6.17209488 3.13428758]
 [4.59654273 6.11166191 6.02924363 6.07090641 3.14648266]
 [4.27511409 6.2837416  5.98534499 5.95801168 3.1380355 ]
 ...
 [3.30319918 5.49826874 6.24239068 5.55535815 3.27583724]
 [4.28934292 5.80630477 6.13589483 6.27387651 3.5027716 ]
 [4.52790166 5.92684055 5.98113295 5.67102763 3.61363472]]


In [ ]:
sqEr_test = 0

original = original.astype(float)
for x in range(TEhalf.shape[0]):
  for y in range(TEhalf.shape[1]):
    if raw_data1[x,y] == "0":
      sqEr_test = sqEr_test + abs(1 - TEhalf[x,y]/original[x,y])
      IBCF_error_doc.append(1 - TEhalf[x,y]/original[x,y])
print(sqEr_test)

71.8486520455676


In [ ]:
print(len(IBCF_error_doc))

1148


In [ ]:
from google.colab import files

file1_name = "IBCF_1415_doc.txt"
IBCF_error_doc = [str(item) for item in IBCF_error_doc]

with open(file1_name, 'w') as file:
  for item in IBCF_error_doc:
        file.write(item + '\n')

files.download(file1_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
np.set_printoptions(threshold=10)
print(raw_data1)

[['6.119271599' '5.905514512' '6.456711355' '2.160587049']
 ['5.855879172' '7.118312419' '7.364973928' '3.616688289']
 ['6.434748022' '6.790187269' '7.058421111' '2.843622473']
 ...
 ['6.271447065' '6.201478176' '6.467272333' '3.011499335']
 ['5.530375459' '6.021418903' '6.778383726' '3.001082395']
 ['6.00129518' '6.353492986' '6.332680103' '3.355064895']]
